# Mixing Station Workspace

Update the .csv file to include the desired constituent electrolytes and their volume ratios, making sure they match up to the correct numbered location on the physical workspace. The columns in the .csv file correspond to the following.. 

| Name | Volume (uL) | Starting Volume (mL) | Aspirate Speed (W/s) | Aspirate Constant (mbar/ml) |
| --- | --- | --- | --- | --- |

Run the code block below and confirm that your total volume is correct.

In [1]:
from src.robot_controller import experiment_setup
experiment = experiment_setup.experiment("electrolye_recipe.csv", "COM12", "COM11", SIM=True)

INFO:root:No serial connection to gantry established.


AttributeError: module 'robot_controller.pipette_controller' has no attribute 'pipette'

## Run Experiment

In [2]:
experiment.run()

INFO:root:Moving to O2..
INFO:root:Pipette charged.
INFO:root:Dropping Pipette into O2..
INFO:root:Aspiration complete.
INFO:root:Moving Pipette out of O2..
INFO:root:Moving to Mixing Chamber..
INFO:root:Dispense complete.
INFO:root:Moving to N2..
INFO:root:Pipette charged.
INFO:root:Dropping Pipette into N2..
INFO:root:Aspiration complete.
INFO:root:Moving Pipette out of N2..
INFO:root:Moving to Mixing Chamber..
INFO:root:Dispense complete.
INFO:root:Moving to CH4..
INFO:root:Pipette charged.
INFO:root:Dropping Pipette into CH4..
INFO:root:Aspiration complete.
INFO:root:Moving Pipette out of CH4..
INFO:root:Moving to Mixing Chamber..
INFO:root:Dispense complete.
INFO:root:Moving to NH4..
INFO:root:Pipette charged.
INFO:root:Dropping Pipette into NH4..
INFO:root:Aspiration complete.
INFO:root:Moving Pipette out of NH4..
INFO:root:Moving to Mixing Chamber..
INFO:root:Dispense complete.
INFO:root:Moving to CH3CH3..
INFO:root:Pipette charged.
INFO:root:Dropping Pipette into CH3CH3..
INFO: